In [72]:
import cv2
import numpy as np
import random
from matplotlib import pyplot as plt
from snake.imagering import trasitionExtend
from components.layer import Layer
from components.network import Network
from components.activations import logsig, satlins

In [2]:
random.seed(123)

In [116]:
%matplotlib inline

def printBoard(board, i = 0):
    filename = "video/" + str(i).rjust(10, "0") + ".jpg"
    plt.ioff()
    plt.figure(figsize=(8, 8))
    plt.axis('off')
    plt.imshow(board, cmap='gray', vmin=0, vmax=1)
    plt.savefig(filename, bbox_inches='tight')
    plt.close()
    # plt.show()
    # im = Image.fromarray(board)
    # im = im.convert('RGB')
    # save_filename=filename
    # im.save(save_filename, "JPEG")

In [ ]:
board = np.zeros((50,50),dtype=np.float32)

In [ ]:
board[5, 3] = 0.5
board[31, 3:15] = 1
printBoard(board)

In [124]:
class Snake:
    LEFT, RIGHT, UP, DOWN = [(0,1), (0,-1), (-1,0), (1,0)]

    def __init__(self, brain) -> None:
        self._body = [(24, 24)]
        self._dir = self.LEFT
        self.total_moves = 0
        self.total_grows = 0
        self.brain = brain
    
    def move(self, grow=False) -> bool: # return if had collision
        head = self._body[0] # head positions
        newHead = (head[0] + self._dir[0], head[1] + self._dir[1])

        if ( newHead[0] < 0 or newHead [0] > 49):
            return True # collision
        if ( newHead[1] < 0 or newHead [1] > 49):
            return True # collision
        
        for part in self._body:
            if(newHead == part):
                return True # collision
        
        self._body.insert(0, newHead)
        if not grow:
            self._body.pop()
        return False
    
    def think(self, board):
        block_distances = []
        food_distances = []

        for i in range(-1, 2):
            for j in range(-1, 2):
                if j == i == 0:
                    continue
                block_distances.append(self._calc_dis(i, j, 1, board)) 
                food_distances.append(self._calc_dis(i, j, 0.5, board, True))

        tail_dir = self._dir
        if(self.total_grows > 0):
            prev= self._body[-2]
            tail = self._body[-1]
            tail_dir = ( prev[0] - tail[0], prev[1] - tail[1] )
        
        food_distances = food_distances / np.linalg.norm(food_distances)
        block_distances = block_distances / np.linalg.norm(block_distances)
        input = [*food_distances, *block_distances, *self._dir, *tail_dir, self.total_grows ]
        output = self.brain.activate(input)
        index = 0
        max = output[0]
        for i in range(1, 4):
            if (output[i] > max):
                max = output[i]
                index = i
        if index == 0:
            self._dir = self.LEFT
        elif index == 1:
            self._dir = self.RIGHT
        elif index == 2:
            self._dir = self.UP
        else:
            self._dir = self.DOWN
    
    def _calc_dis(self, x, y, goal, board, hundredIfNotFound = False):
        i, j = self.getHead()
        dist = 0
        found = False

        while (i < 49 and j < 49 and i > 0 and j > 0):
            i += x
            j += y
            dist += 1
            if(board[i][j] == goal):
                found = True
                break
        if found or not hundredIfNotFound: 
            return dist
        return 100
        
        

    
    def setDir(self, dir):
        sum = self._dir[0] + dir[0] + self._dir[1] + dir[1]
        if sum != 0:
            self._dir = dir
    
    def getHead(self):
        return self._body[0]
    
    def print(self, board):
        for part in self._body:
            board[part[0], part[1]] = 1


In [98]:
x= (1, 2)
[*x, 3]

[1, 2, 3]

In [ ]:
board = np.zeros((50,50),dtype=np.float32)
collision = False
snake = Snake()
steps = []

while not collision:
    board.fill(0)
    collision = snake.move()
    snake.print(board)
    steps.append(board.copy())

for i in range(len(steps)):
    printBoard(steps[i], i)
        
# ffmpeg -framerate 30 -pattern_type glob -i "*.jpg" output.avi

### Brain 1
- 8 distances food in the direction (negative if there insn't)
- 8 distances to obstruction
- 2 current head direction
- 2 current tail direction
- 1 current size

i:21x1 -> h:21x21 -> h:14x21 -> o:4x14 -> 4x1

In [119]:
def brain1():
    return Network([
        Layer(
            weights = (np.random.rand(21, 21) - 0.5),
            biases = np.random.rand(21, 1) - 0.5,
            activations=satlins
        ),
        Layer(
            weights = (np.random.rand(14, 21) - 0.5),
            biases = np.random.rand(14, 1),
            activations=satlins
        ),
        Layer(
            weights = (np.random.rand(4, 14) - 0.5) * -2,
            biases = np.random.rand(4, 1),
            activations=logsig
        )
    ])

In [150]:
board = np.zeros((50,50),dtype=np.float32)
collision = False
snake = Snake(brain1())
steps = []
food_pos = (5, 25)
eaten_food = False

def rand_food(board, depth = 0):
    if(depth > 30):
        exit(1)
    food_pos = (random.randint(0, 49), random.randint(0, 49))
    if(board[food_pos[0]][food_pos[1]] == 1):
        return rand_food(board, depth + 1)
    return food_pos

while not collision:
    board.fill(0)
    board[food_pos[0], food_pos[1]] = 0.5
    collision = snake.move(grow=eaten_food)
    snake.print(board)
    eaten_food = food_pos == snake.getHead()
    if eaten_food:
        food_pos = rand_food(board)
        board[food_pos[0], food_pos[1]] = 0.5
    snake.think(board)
    steps.append(board.copy())

# for i in range(len(steps)):
#     printBoard(steps[i], i)